In [1]:
# I want to confirm the idea of optimization of unitary matrix
import sys
sys.path.insert(0,"../")

In [2]:
import numpy as np
import rms
import jax
import jax.numpy as jnp

/home/user/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# generate random hamiltonian 
n = 4 # local dimension
N = n*n

### Random Hamiltonian sample from normal distribution

In [35]:
H = np.random.normal(size = (N,N))
H = (H + H.T)/2
H = jnp.array(H)
e = jnp.linalg.eigvalsh(rms.stoquastic(U @ H @ U.T))[0]
e_min = jnp.linalg.eigvalsh(U @ H @ U.T)[0]
print("initial loss : ", -e)
print("minimum loss : ", -e_min)
state = rms.loss.init_loss(H, n, np.float64, "smel")
state_list = [state]
qesLoss = rms.loss.system_mel_multi
lion_solver = rms.solver.lionSolver(qesLoss, state_list, system = True)
seed = np.random.randint(0, 100000)
np.random.seed(seed)
ur = rms.unitary.UnitaryRiemanGenerator(n, jax.random.PRNGKey(seed), np.float64)

initial loss :  9.205718966887204
minimum loss :  5.243969924646554
UnitaryRiemanGenerator is initialized


In [36]:
u = ur.reset_matrix()
u, lv = lion_solver(
    u,
    1000,
    0.001,
    cout=True,
    cutoff_cnt=10,
    mass1=0.9,
    mass2=0.6,
    offset=0.001
)

U = jnp.kron(u,u)
HU = U @ H @ U.T
e_ = jnp.linalg.eigvalsh(rms.stoquastic(HU))[0]
print("optimized loss : ", -e_)

 40%|█████████████▉                     | 399/1000 [00:03<00:05, 110.33it/s, iter=399, lr = 0.00100 bad_cnt=10, loss=8.43067, ]

optimized loss :  8.430168137513581


In [37]:
jnp.linalg.eigvalsh(rms.stoquastic(HU))

Array([-8.43016814, -3.7156706 , -2.63084349, -2.35976217, -1.13788208,
       -0.87965537, -0.53801147,  0.00876012,  0.17746265,  0.37446924,
        1.05378425,  1.55344456,  1.7663003 ,  2.63530019,  3.58333481,
        4.6169361 ], dtype=float64)

In [39]:
np.round(HU - rms.stoquastic(HU),5)

Array([[ 0.     ,  3.16797,  0.     ,  0.51317,  0.80946,  0.     ,
         0.     ,  0.1354 ,  1.06842,  0.     ,  0.     ,  1.56992,
         0.11966,  0.     ,  0.     ,  0.     ],
       [ 3.16797,  0.     ,  0.0371 ,  0.28155,  0.     ,  0.31971,
         0.     ,  0.     ,  0.     ,  1.92212,  0.18022,  0.     ,
         0.90578,  1.25505,  0.26281,  1.50342],
       [ 0.     ,  0.0371 , -0.     ,  1.16065,  0.     ,  1.70787,
         0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  1.21447,
         0.     ,  0.     ,  0.     ,  0.05527],
       [ 0.51317,  0.28155,  1.16065,  0.     ,  0.4075 ,  0.47478,
         0.     ,  0.     ,  0.0074 ,  1.05954,  0.     ,  3.23323,
         0.     ,  0.     ,  0.     ,  3.62089],
       [ 0.80946,  0.     ,  0.     ,  0.4075 ,  0.     ,  3.0001 ,
         0.     ,  0.35123,  1.61242,  0.03759,  3.5516 ,  2.50922,
         0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.31971,  1.70787,  0.47478,  3.0001 ,  0.     ,
       